### Notes

* changed shape[1] from 4 to 5

In [1]:
%matplotlib inline

In [2]:
import sys
import numpy as np
import tensorflow as tf

from batcher import Batcher
from metrics import acc012

In [3]:
X_train = np.load('../../data/bin/train/deal.npy')
y_train = np.load('../../data/bin/train/tricks_notrump.npy')

X_val = np.load('../../data/bin/val/deal.npy')
y_val = np.load('../../data/bin/val/tricks_notrump.npy')

n_examples = X_train.shape[0]

n_h = X_train.shape[1]
n_w = X_train.shape[2]
n_c = X_train.shape[3]

batch_size = 64
n_iterations = 2000000
display_step = 1000

learning_rate = 0.001

n_hidden_units = 128

l2_reg = 0.05

In [4]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((10800000, 4, 13, 4), (100000, 4, 13, 4), (10800000, 1), (100000, 1))

In [5]:
X = tf.placeholder(tf.float32, shape=[None, n_h, n_w, n_c])
Y = tf.placeholder(tf.float32, shape=[1, None])
strides = [1,1,1,1]

conv1_w = tf.get_variable('c1w', shape=[1, 5, 4, 32], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv1_z = tf.nn.conv2d(X, filter=conv1_w, strides=strides, padding='SAME')
conv1_a = tf.nn.relu(conv1_z)

conv2_w = tf.get_variable('c2w', shape=[1, 5, 32, 64], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv2_z = tf.nn.conv2d(conv1_a, filter=conv2_w, strides=strides, padding='SAME')
conv2_a = tf.nn.relu(conv2_z)

conv3_w = tf.get_variable('c3w', shape=[1, 5, 64, 128], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv3_z = tf.nn.conv2d(conv2_a, filter=conv3_w, strides=strides, padding='SAME')
conv3_a = tf.nn.relu(conv3_z)

conv4_w = tf.get_variable('c4w', shape=[4, 5, 128, 512], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv4_z = tf.nn.conv2d(conv3_a, filter=conv4_w, strides=strides, padding='VALID')
conv4_a = tf.nn.relu(conv4_z)

fc_in = tf.contrib.layers.flatten(conv4_a)
fc_w = tf.get_variable('fcw', shape=[n_hidden_units, fc_in.shape.as_list()[1]], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
fc_b = tf.Variable(np.zeros((n_hidden_units, 1)), dtype=tf.float32)
fc_z = tf.add(tf.matmul(fc_w, tf.transpose(fc_in)), fc_b)
fc_a = tf.nn.relu(fc_z)

w_out = tf.get_variable('w_out', shape=[1, n_hidden_units], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
b_out = tf.Variable(np.zeros((1, 1)), dtype=tf.float32)
pred = tf.add(tf.matmul(w_out, fc_a), b_out)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [6]:
cost = tf.reduce_mean(tf.squared_difference(pred, Y))

In [7]:
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9).minimize(cost)

In [8]:
init = tf.global_variables_initializer()

In [9]:
batch = Batcher(n_examples, batch_size)
cost_train_batch = Batcher(n_examples, 10000)
cost_val_batch = Batcher(100000, 10000)

In [10]:
costs = []

with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(n_iterations):
        x_batch, y_batch = batch.next_batch([X_train, y_train])
        
        if iteration % display_step == 0:
            sys.stdout.write('*')
            x_batch_c, y_batch_c = cost_train_batch.next_batch([X_train, y_train])
            x_batch_v, y_batch_v = cost_val_batch.next_batch([X_val, y_val])
            c = sess.run(cost, feed_dict={X: x_batch_c, Y: y_batch_c.T})
            costs.append(c)
            pred_train = sess.run(pred, feed_dict={X: x_batch_c, Y: y_batch_c.T})
            pred_val = sess.run(pred, feed_dict={X: x_batch_v, Y: y_batch_v.T})
            print('it={} cost={}'.format(iteration, c))
            print(acc012(y_batch_c, pred_train.T))
            print(acc012(y_batch_v, pred_val.T))
            
        sess.run(optimizer, feed_dict={X:x_batch, Y:y_batch.T})
    
    # Testing
    graph = tf.get_default_graph()
    
    X_test = np.load('../../data/bin/test/deal.npy')
    y_test = np.load('../../data/bin/test/tricks_notrump.npy')
    
    print('\n\nTesting')
    predictions = sess.run(pred, feed_dict = {X:X_test})

    print(acc012(y_test.ravel(), predictions.round().ravel()))

*it=0 cost=43.34741973876953
(0.018, 0.0533, 0.1133)
(0.0165, 0.0545, 0.115)
*it=1000 cost=1.5245331525802612
(0.3136, 0.7804, 0.9557)
(0.3144, 0.7755, 0.9538)
*it=2000 cost=1.5277372598648071
(0.3326, 0.7977, 0.9531)
(0.3327, 0.8048, 0.9561)
*it=3000 cost=1.296378493309021
(0.3635, 0.8256, 0.9666)
(0.3543, 0.8301, 0.9683)
*it=4000 cost=1.1591124534606934
(0.3724, 0.8486, 0.9728)
(0.3777, 0.8453, 0.9747)
*it=5000 cost=1.210432767868042
(0.3677, 0.8421, 0.971)
(0.3657, 0.8429, 0.9735)
*it=6000 cost=1.179775357246399
(0.3768, 0.8425, 0.9717)
(0.3759, 0.8409, 0.9721)
*it=7000 cost=1.065140962600708
(0.4007, 0.8683, 0.9775)
(0.389, 0.8676, 0.9771)
*it=8000 cost=1.0746138095855713
(0.4028, 0.8663, 0.9762)
(0.405, 0.8664, 0.9775)
*it=9000 cost=0.9885134696960449
(0.4164, 0.8812, 0.9799)
(0.4198, 0.877, 0.981)
*it=10000 cost=0.9989900588989258
(0.4122, 0.8742, 0.9798)
(0.4131, 0.871, 0.9807)
*it=11000 cost=0.9807526469230652
(0.4145, 0.8745, 0.9826)
(0.4127, 0.8815, 0.9812)
*it=12000 cost=0.9

*it=99000 cost=0.481113076210022
(0.5771, 0.9625, 0.9958)
(0.5849, 0.9616, 0.9962)
*it=100000 cost=0.45730826258659363
(0.5931, 0.966, 0.9966)
(0.5876, 0.9636, 0.9967)
*it=101000 cost=0.46790874004364014
(0.5881, 0.9643, 0.9966)
(0.5932, 0.9642, 0.997)
*it=102000 cost=0.47015389800071716
(0.5839, 0.9621, 0.9964)
(0.5867, 0.9641, 0.9961)
*it=103000 cost=0.4631583094596863
(0.5941, 0.9638, 0.9963)
(0.5772, 0.9594, 0.9945)
*it=104000 cost=0.4568198621273041
(0.5878, 0.9656, 0.9973)
(0.587, 0.9623, 0.9973)
*it=105000 cost=0.4524635672569275
(0.601, 0.965, 0.9956)
(0.6046, 0.9651, 0.9968)
*it=106000 cost=0.44200170040130615
(0.596, 0.9669, 0.9974)
(0.5986, 0.9658, 0.996)
*it=107000 cost=0.45223715901374817
(0.5935, 0.9659, 0.9972)
(0.5952, 0.9648, 0.9964)
*it=108000 cost=0.45224297046661377
(0.5986, 0.9652, 0.9953)
(0.5959, 0.9677, 0.9968)
*it=109000 cost=0.4304628074169159
(0.6063, 0.9696, 0.9969)
(0.6002, 0.9674, 0.9965)
*it=110000 cost=0.501838743686676
(0.567, 0.9608, 0.9958)
(0.5753, 0

*it=196000 cost=0.3484591245651245
(0.6553, 0.9792, 0.9982)
(0.6574, 0.9767, 0.9979)
*it=197000 cost=0.3457222282886505
(0.6606, 0.9781, 0.998)
(0.6519, 0.9768, 0.9979)
*it=198000 cost=0.3636343777179718
(0.6473, 0.9781, 0.9982)
(0.6321, 0.976, 0.9974)
*it=199000 cost=0.34531593322753906
(0.6491, 0.9805, 0.9988)
(0.6438, 0.9778, 0.9977)
*it=200000 cost=0.3550513684749603
(0.6639, 0.977, 0.9977)
(0.6475, 0.9731, 0.9973)
*it=201000 cost=0.35587090253829956
(0.6456, 0.9793, 0.9989)
(0.6433, 0.9764, 0.998)
*it=202000 cost=0.3617517650127411
(0.6591, 0.9772, 0.9974)
(0.6615, 0.9785, 0.9985)
*it=203000 cost=0.36414840817451477
(0.6449, 0.9793, 0.9973)
(0.6451, 0.9782, 0.9984)
*it=204000 cost=0.3338930606842041
(0.6633, 0.9818, 0.9982)
(0.6514, 0.9784, 0.9984)
*it=205000 cost=0.3502067029476166
(0.6621, 0.9787, 0.9979)
(0.6435, 0.9769, 0.9978)
*it=206000 cost=0.347790002822876
(0.6633, 0.9782, 0.9978)
(0.6469, 0.9755, 0.9977)
*it=207000 cost=0.3404289484024048
(0.6586, 0.9817, 0.9982)
(0.6501

*it=293000 cost=0.3072303831577301
(0.6823, 0.9832, 0.9984)
(0.6883, 0.9812, 0.9976)
*it=294000 cost=0.30884304642677307
(0.682, 0.9835, 0.9991)
(0.6838, 0.9805, 0.9984)
*it=295000 cost=0.31885796785354614
(0.681, 0.9814, 0.9983)
(0.6904, 0.9811, 0.9983)
*it=296000 cost=0.31629326939582825
(0.6932, 0.9821, 0.9982)
(0.6779, 0.9818, 0.9982)
*it=297000 cost=0.3116057813167572
(0.6806, 0.9839, 0.9984)
(0.6823, 0.9826, 0.9984)
*it=298000 cost=0.32371509075164795
(0.6811, 0.9819, 0.9978)
(0.675, 0.9809, 0.9984)
*it=299000 cost=0.30361685156822205
(0.6933, 0.9826, 0.9986)
(0.6877, 0.9823, 0.9987)
*it=300000 cost=0.298132985830307
(0.6971, 0.9832, 0.9987)
(0.6925, 0.9841, 0.9985)
*it=301000 cost=0.31476080417633057
(0.6897, 0.9815, 0.9979)
(0.6779, 0.9823, 0.9982)
*it=302000 cost=0.31687596440315247
(0.6834, 0.9819, 0.9988)
(0.681, 0.9829, 0.9989)
*it=303000 cost=0.3289514482021332
(0.6729, 0.9806, 0.9983)
(0.6682, 0.9806, 0.9986)
*it=304000 cost=0.2971895635128021
(0.6898, 0.9846, 0.9989)
(0.

*it=390000 cost=0.2996498644351959
(0.7002, 0.9852, 0.9983)
(0.6943, 0.9829, 0.999)
*it=391000 cost=0.28305503726005554
(0.7021, 0.9871, 0.999)
(0.7028, 0.9826, 0.9987)
*it=392000 cost=0.2784580588340759
(0.7143, 0.9869, 0.9993)
(0.7032, 0.9858, 0.9986)
*it=393000 cost=0.2931145131587982
(0.7042, 0.985, 0.9982)
(0.7018, 0.9838, 0.999)
*it=394000 cost=0.2881752550601959
(0.7006, 0.9861, 0.9989)
(0.698, 0.9851, 0.9981)
*it=395000 cost=0.2843623459339142
(0.707, 0.9856, 0.9991)
(0.7015, 0.9858, 0.999)
*it=396000 cost=0.2769443988800049
(0.7113, 0.9861, 0.9989)
(0.7041, 0.9846, 0.9987)
*it=397000 cost=0.2822292447090149
(0.7055, 0.9872, 0.9989)
(0.7001, 0.9851, 0.9988)
*it=398000 cost=0.2888014018535614
(0.7066, 0.9851, 0.9987)
(0.6969, 0.9834, 0.9989)
*it=399000 cost=0.27662548422813416
(0.714, 0.9868, 0.9988)
(0.695, 0.9829, 0.9984)
*it=400000 cost=0.2757839262485504
(0.7006, 0.9892, 0.9992)
(0.707, 0.9852, 0.9989)
*it=401000 cost=0.2860158681869507
(0.7066, 0.9852, 0.999)
(0.7029, 0.985

...

*it=1548000 cost=0.19327005743980408
(0.777, 0.9949, 0.9996)
(0.7527, 0.9902, 0.9992)
*it=1549000 cost=0.18839503824710846
(0.7827, 0.9951, 0.9999)
(0.7538, 0.9902, 0.9997)
*it=1550000 cost=0.2053123563528061
(0.7708, 0.993, 0.9996)
(0.7528, 0.99, 0.9991)
*it=1551000 cost=0.1864643543958664
(0.7876, 0.9956, 0.9998)
(0.7631, 0.9909, 0.9996)
*it=1552000 cost=0.18527045845985413
(0.7917, 0.9944, 0.9998)
(0.7554, 0.9924, 0.9994)
*it=1553000 cost=0.19560649991035461
(0.784, 0.9938, 0.9993)
(0.7623, 0.9911, 0.9995)
*it=1554000 cost=0.2014838606119156
(0.7752, 0.9932, 0.9996)
(0.7558, 0.9912, 0.9994)
*it=1555000 cost=0.19606970250606537
(0.7811, 0.9933, 0.9996)
(0.7629, 0.992, 0.999)
*it=1556000 cost=0.19151294231414795
(0.7812, 0.9959, 0.9996)
(0.7612, 0.9926, 0.9993)
*it=1557000 cost=0.19638508558273315
(0.7818, 0.9935, 0.9996)
(0.7643, 0.993, 0.9992)
*it=1558000 cost=0.18993952870368958
(0.7834, 0.9948, 0.9997)
(0.7605, 0.9913, 0.9992)
*it=1559000 cost=0.19705970585346222
(0.777, 0.9948, 0

*it=1643000 cost=0.1988593190908432
(0.78, 0.9936, 0.9994)
(0.7651, 0.9919, 0.9994)
*it=1644000 cost=0.1915675550699234
(0.7832, 0.9945, 0.9999)
(0.7587, 0.992, 0.9994)
*it=1645000 cost=0.1928606629371643
(0.7821, 0.9948, 0.9997)
(0.7623, 0.9929, 0.9993)
*it=1646000 cost=0.19160854816436768
(0.7848, 0.9946, 0.9996)
(0.7643, 0.9934, 0.9995)
*it=1647000 cost=0.1916741579771042
(0.7807, 0.9955, 0.9998)
(0.7599, 0.9903, 0.9993)
*it=1648000 cost=0.18721428513526917
(0.7887, 0.9947, 0.9999)
(0.756, 0.9921, 0.9991)
*it=1649000 cost=0.19287486374378204
(0.779, 0.9946, 0.9995)
(0.7635, 0.9904, 0.9995)
*it=1650000 cost=0.1875525861978531
(0.792, 0.9954, 0.9997)
(0.7565, 0.992, 0.9992)
*it=1651000 cost=0.18839187920093536
(0.7892, 0.9947, 0.9995)
(0.7674, 0.9915, 0.9998)
*it=1652000 cost=0.18958669900894165
(0.7884, 0.9948, 0.9996)
(0.7664, 0.9911, 0.9996)
*it=1653000 cost=0.1906537413597107
(0.7855, 0.9942, 0.9995)
(0.762, 0.9922, 0.9996)
*it=1654000 cost=0.1833963841199875
(0.7893, 0.9952, 0.99

*it=1739000 cost=0.1904275268316269
(0.782, 0.9947, 0.9997)
(0.7579, 0.9916, 0.9993)
*it=1740000 cost=0.19063685834407806
(0.7827, 0.9942, 0.9998)
(0.7602, 0.9932, 0.9998)
*it=1741000 cost=0.1962377429008484
(0.7787, 0.9938, 0.9998)
(0.7641, 0.9925, 0.9995)
*it=1742000 cost=0.18261359632015228
(0.7965, 0.9948, 0.9998)
(0.7591, 0.9909, 0.9993)
*it=1743000 cost=0.19409841299057007
(0.7853, 0.9937, 0.9995)
(0.7651, 0.9915, 0.9992)
*it=1744000 cost=0.1877855658531189
(0.7864, 0.9941, 0.9999)
(0.7608, 0.9922, 0.9996)
*it=1745000 cost=0.1882970631122589
(0.7932, 0.9952, 0.9997)
(0.7626, 0.9923, 0.9995)
*it=1746000 cost=0.19084836542606354
(0.7887, 0.9943, 0.9996)
(0.7601, 0.9915, 0.9988)
*it=1747000 cost=0.1888972967863083
(0.7928, 0.9931, 0.9997)
(0.768, 0.991, 0.9988)
*it=1748000 cost=0.18452705442905426
(0.789, 0.9956, 0.9998)
(0.7614, 0.9932, 0.9997)
*it=1749000 cost=0.1868618130683899
(0.7834, 0.9945, 1.0)
(0.7645, 0.9928, 0.9998)
*it=1750000 cost=0.19024775922298431
(0.7835, 0.9937, 0.

*it=1835000 cost=0.18451666831970215
(0.7949, 0.9945, 0.9999)
(0.7675, 0.9932, 0.9994)
*it=1836000 cost=0.18993119895458221
(0.7871, 0.9942, 0.9996)
(0.7681, 0.9903, 0.9989)
*it=1837000 cost=0.18354260921478271
(0.7907, 0.9959, 0.9998)
(0.7612, 0.9915, 0.9992)
*it=1838000 cost=0.18909378349781036
(0.7862, 0.9949, 0.9998)
(0.7651, 0.9917, 0.9992)
*it=1839000 cost=0.18534792959690094
(0.7891, 0.9949, 0.9999)
(0.7618, 0.9917, 0.9993)
*it=1840000 cost=0.19088171422481537
(0.783, 0.9947, 0.9997)
(0.766, 0.9933, 0.9996)
*it=1841000 cost=0.18393130600452423
(0.7842, 0.9953, 0.9997)
(0.763, 0.993, 0.9993)
*it=1842000 cost=0.18937167525291443
(0.7876, 0.994, 0.9998)
(0.7657, 0.9917, 0.9992)
*it=1843000 cost=0.19143857061862946
(0.787, 0.994, 0.9995)
(0.7687, 0.992, 0.9995)
*it=1844000 cost=0.18433436751365662
(0.7932, 0.995, 0.9999)
(0.7668, 0.9916, 0.9993)
*it=1845000 cost=0.1913723200559616
(0.7806, 0.9949, 0.9999)
(0.7624, 0.992, 0.9994)
*it=1846000 cost=0.1882629543542862
(0.7895, 0.994, 0.

*it=1931000 cost=0.17870214581489563
(0.7942, 0.9957, 0.9998)
(0.7603, 0.9911, 0.9995)
*it=1932000 cost=0.18136753141880035
(0.7943, 0.9945, 1.0)
(0.7739, 0.9935, 0.9996)
*it=1933000 cost=0.18423713743686676
(0.7865, 0.9954, 0.9999)
(0.7608, 0.9916, 0.9995)
*it=1934000 cost=0.18368138372898102
(0.7943, 0.995, 0.9998)
(0.7706, 0.9918, 0.9994)
*it=1935000 cost=0.17934298515319824
(0.7942, 0.9951, 0.9999)
(0.766, 0.9913, 0.9994)
*it=1936000 cost=0.1817426234483719
(0.794, 0.9957, 0.9997)
(0.7665, 0.9917, 0.9995)
*it=1937000 cost=0.18031111359596252
(0.7937, 0.9959, 0.9997)
(0.7682, 0.9936, 0.9995)
*it=1938000 cost=0.18295444548130035
(0.792, 0.9948, 0.9999)
(0.7624, 0.9929, 0.9996)
*it=1939000 cost=0.17954400181770325
(0.7951, 0.9957, 0.9999)
(0.7686, 0.9914, 0.9989)
*it=1940000 cost=0.18588942289352417
(0.7918, 0.9948, 0.9997)
(0.7641, 0.9905, 0.9993)
*it=1941000 cost=0.1816079169511795
(0.787, 0.9953, 0.9997)
(0.7633, 0.9925, 0.9999)
*it=1942000 cost=0.1853177398443222
(0.7862, 0.9949, 

In [1]:
# import pandas as pd
# path = '../../bridgent/preds/z_v2_preds.pickle'
# pd.to_pickle({'preds':predictions, 'acc012':acc012(y_test.ravel(), predictions.round().ravel())}, path)

NameError: name 'predictions' is not defined

In [3]:
import pandas as pd
path = '../../bridgent/preds/z_v2_preds.pickle'
res = pd.read_pickle(path)
res

{'preds': array([[5.003008 , 3.1861954, 7.1094666, ..., 5.2906675, 5.882684 ,
         7.958911 ]], dtype=float32),
 'acc012': (0.76442, 0.99159, 0.99928)}

In [5]:
res['preds'].shape

(1, 100000)